# Chapter 1 - Manim in Jupyter

Working with manim in jupyter notebooks has servel advantages:
* outputs and code are close together
* easy to iterate examples
* computation intensive code can be executed serperately from the scenes
* global Mobjects can be used in multiple scenes.
* more can bee seen in chapter 4

In [ ]:
from manim import * 

In [ ]:
params = "-v WARNING  -s -ql --disable_caching Example"

In [ ]:
%%manim $params
class Example(Scene):
    def construct(self):
        c = Circle(stroke_width=30)
        self.add(c)

In [ ]:
%%time
np.random.seed(20)
steps = np.random.choice(a=[-1, 0, 1], size=(100000,1000))
stop = steps.cumsum(0)
end_points= stop[-1]/stop[-1].max() 
end_pointsX = end_points[0:500]
end_pointsY = end_points[500:-1]

In [ ]:
%%time
%%manim $params
class Example(Scene):
    def construct(self):
        dots = [Dot(point=[3*x,3*y,0]).scale(0.7) for  x,y in zip(end_pointsX, end_pointsY)]
        self.add(*dots)